In [42]:
import pandas as pd
import datetime

In [64]:
df_record = pd.read_csv("s3://applehealthdashboard/etl/extract/exportRecord.csv")
df_workout = pd.read_csv("s3://applehealthdashboard/etl/extract/exportWorkout.csv")
df_summary = pd.read_csv("s3://applehealthdashboard/etl/extract/exportActivitySummary.csv")

/var/folders/23/bwtd0x3j2qjf60tdmj7g56400000gn/T/ipykernel_4058/2773105388.py:1: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_record = pd.read_csv("s3://applehealthdashboard/etl/extract/exportRecord.csv")


In [53]:
df_workout_test = df_workout.rename(columns={"workoutActivityType": "type", "duration": "value", "durationUnit": "unit"})

In [86]:
df_test = df_record.copy()
type_prefixes = {
    "HKCategoryTypeIdentifier": "category",
    "HKQuantityTypeIdentifier": "quantity",
    "HKWorkoutActivityType": "workout",
}
def parse_category(activity_type: str) -> str:
        for key in type_prefixes:
            if key in activity_type:
                return key
df_test["category_prefix"] = df_test["type"].apply(parse_category)

df_test["type"] = df_test["type"].str.replace(
        "|".join(list(type_prefixes.keys())), "", regex=True
    )

In [87]:
df_test

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device,category_prefix
0,Height,iPhone (7),15.5,ft,2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,5.91667,NaN,HKQuantityTypeIdentifier
1,BodyMass,iPhone (7),15.5,lb,2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,2022-07-04 08:41:18 -0700,150.0,NaN,HKQuantityTypeIdentifier
2,BodyMass,iPhone (7),16.3,lb,2023-05-07 13:35:43 -0700,2023-05-07 13:35:43 -0700,2023-05-07 13:35:43 -0700,150.0,NaN,HKQuantityTypeIdentifier
3,HeartRate,Eric’s Apple Watch,8.3,count/min,2022-07-04 08:45:31 -0700,2022-07-04 08:43:53 -0700,2022-07-04 08:43:53 -0700,50.0,"<<HKDevice: 0x2832c9130>, name:Apple Watch, ma...",HKQuantityTypeIdentifier
4,HeartRate,Eric’s Apple Watch,8.3,count/min,2022-07-04 08:47:12 -0700,2022-07-04 08:47:15 -0700,2022-07-04 08:47:15 -0700,53.0,"<<HKDevice: 0x2832c9130>, name:Apple Watch, ma...",HKQuantityTypeIdentifier
...,...,...,...,...,...,...,...,...,...,...
1711275,HeartRateVariabilitySDNN,Eric’s Apple Watch,10.2,ms,2024-03-17 02:09:02 -0700,2024-03-17 02:08:01 -0700,2024-03-17 02:09:00 -0700,39.1339,"<<HKDevice: 0x283255b80>, name:Apple Watch, ma...",HKQuantityTypeIdentifier
1711276,HeartRateVariabilitySDNN,Eric’s Apple Watch,10.2,ms,2024-03-17 04:09:00 -0700,2024-03-17 04:07:59 -0700,2024-03-17 04:08:59 -0700,429.662,"<<HKDevice: 0x283255b80>, name:Apple Watch, ma...",HKQuantityTypeIdentifier
1711277,HeartRateVariabilitySDNN,Eric’s Apple Watch,10.2,ms,2024-03-17 06:09:05 -0700,2024-03-17 06:08:04 -0700,2024-03-17 06:09:04 -0700,54.8017,"<<HKDevice: 0x283255b80>, name:Apple Watch, ma...",HKQuantityTypeIdentifier
1711278,HeartRateVariabilitySDNN,Eric’s Apple Watch,10.2,ms,2024-03-17 08:10:45 -0700,2024-03-17 08:09:44 -0700,2024-03-17 08:10:43 -0700,56.8167,"<<HKDevice: 0x283255b80>, name:Apple Watch, ma...",HKQuantityTypeIdentifier


In [69]:
import pandas as pd
import re

# Step 1: Define a function to parse the string
def parse_device_string(device_string):
    if pd.isnull(device_string):
        return {}
    device_string = device_string[1:-1]  # remove the angle brackets
    device_string = device_string.split(", ")
    device_dict = {}
    for item in device_string:
        key, value = item.split(":", 1)  # only split on the first colon
        device_dict[key] = value
    return device_dict

df_test = df_record.copy()

# Step 2: Apply the function to the 'device' column
df_test = df_test.join(pd.json_normalize(df_test['device'].apply(parse_device_string)))


In [75]:
df_record.loc[df_record["type"] == "HKQuantityTypeIdentifierAppleStandHour"]

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device


In [76]:
activity_prefixes = ["HKQuantityTypeIdentifier", "HKCategoryTypeIdentifier", "HKWorkoutActivityType"]
pattern = "|".join(activity_prefixes)

In [80]:
df_workout["workoutActivityType"].str.replace(pattern, '', regex=True).unique()

array(['HKWorkoutActivityTypeStairClimbing',
       'HKWorkoutActivityTypeTraditionalStrengthTraining',
       'HKWorkoutActivityTypeWalking', 'HKWorkoutActivityTypeYoga',
       'HKWorkoutActivityTypeSwimming',
       'HKWorkoutActivityTypeFunctionalStrengthTraining',
       'HKWorkoutActivityTypeHiking', 'HKWorkoutActivityTypeRunning',
       'HKWorkoutActivityTypeHighIntensityIntervalTraining',
       'HKWorkoutActivityTypeRowing', 'HKWorkoutActivityTypeCardioDance',
       'HKWorkoutActivityTypePaddleSports',
       'HKWorkoutActivityTypeElliptical', 'HKWorkoutActivityTypeStairs',
       'HKWorkoutActivityTypePickleball',
       'HKWorkoutActivityTypeBasketball', 'HKWorkoutActivityTypeCricket',
       'HKWorkoutActivityTypeCycling'], dtype=object)

In [88]:
df_record["unit"].unique()

array(['ft', 'lb', 'count/min', '%', 'count', 'mi', 'Cal', 'min', 'yd',
       'mL/min·kg', 'dBASPL', 'm', 'mi/hr', 'in', 'ft/s', 'hr', 'cm',
       'ms', 'W', 'kcal/hr·kg', nan], dtype=object)

In [ ]:
fact_table